# Event display for WbLS data
The Water-based Liquid Scintillator (WbLS) 1-ton detector is a R&D demonstrator for future neutrino or dark matter detectors currently operating at BNL. The WbLS recently developed enables separation between Cherenkov and scintillation signals, thus providing both directional reconstruction and enhanced low-energy efficiency.

![](https://drive.google.com/uc?export=view&id=11Ihqq1x2I5Yk2FFArB6ocmp9_9QyeDni)

This notebook will allow you to access the cosmics data you collected with the WbLS detector and display your events. Since the starting point are raw DAQ files, you will need to import some pre-existing Python modules to unpack them.



## Step 1: Setting up the environment
As always, we start by loading some Python packages that will be useful along the way.
Google Colab provides several pre-installed libraries that can be directly imported. Additional packages can be installed using `!pip install`.

In [ ]:
# these packages are already available in Colab
# so we can import them directly
import os
import sys
import re
import glob
import statistics
import pandas as pd
import types
import numpy as np
from numpy import array, isscalar, uint16, uint32
from numpy import sqrt
from datetime import datetime
from scipy.stats import norm
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt

# these packages require to be installed
!pip install iminuit
from iminuit import Minuit
!pip install uproot
import uproot
!pip install numba
!pip install lmfit
from lmfit import Model

print("uproot version:", uproot.__version__)

The next step is mounting a Google Drive access point, so that data files can be easily accessed. Moreover, this analysis requires a series of pre-written Python modules that will also be stored and accessed via Google Drive.

1.   Run the following cell and follow the instructions in the popup dialog to grant access to the Google Drive connected to your account.
2.   Go to [this Google Drive folder](https://drive.google.com/drive/folders/14SvWSV2c8bupNo5_yVhbXJ_kzsbUB6m_?usp=drive_link) and copy your data files (look for your group name) **and** the `modules` folder in your Google Drive.
4.   Fill the paths to your files/folders in the code. You can also copy the path by esploring the navigation menu on the left side of this page.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# fill out the path to the 'modules' folder in your Google Drive
# e.g.: /content/drive/MyDrive/RENEW_DATA-Week_1/modules
MODULES_PATH = "/content/drive/MyDrive/RENEW_DATA-Week_1/modules"

# fill out the path to your collected data file in your Google Drive
# e.g.: /content/drive/MyDrive/RENEW_DATA-Week_1/sample_data_WbLS/phase2_wbls_1pct_muon_mid_x64_19mV_alpha_2023_0625_0.root
FILE_PATH = "/content/drive/MyDrive/RENEW_DATA-Week_1/sample_data_WbLS/phase2_wbls_1pct_muon_mid_x64_19mV_alpha_2023_0625_0.root"

The following cell uses the paths you provided to setup the pre-written Python modules, so that they can be referenced inside the notebook.
These pre-existing modules come from the [DROP](https://github.com/BNLIF/drop) reconstruction software, developed by those directly working on this R&D system at BNL.

In [ ]:
# setting environment variables
SRC_DIR = MODULES_PATH+"/src"
YAML_DIR = MODULES_PATH+"/yaml"
LIB_DIR = MODULES_PATH+"/lib"
TOOL_DIR = MODULES_PATH+"/tools"
os.environ['LIB_DIR'] = LIB_DIR
os.environ['YAML_DIR'] = YAML_DIR
os.environ['SRC_DIR'] = SRC_DIR
os.environ['TOOL_DIR'] = TOOL_DIR
sys.path.append(SRC_DIR)

# setting up some additional path to calibration and configuration files
CALIB_FILE = MODULES_PATH+"/calib/bnl1t_spe_fit_results_230420T1832.csv"
CONFIG_PATH = MODULES_PATH+"/config_guang.yaml"

# building a libraries of utilities
os.system(MODULES_PATH+"/src/make_numba_lib.py")

# import functions from these python modules
from utilities import generate_colormap
from run_drop import RunDROP
from waveform import Waveform
from yaml_reader import YamlReader
from pulse_finder import PulseFinder
from yaml_reader import SAMPLE_TO_NS

## Step 2: Extracting data
The following cells contain functions to read-out data from the file. These function are organized in a *class*, called `WaveformExtractor`, which extracts both metadata and channel by channel information.

In particular, for every PMT readout channel, two quantities are extracted by analyzing the waveform:
*   The total collected charge in photoelectrons (`pe`), via `get_wfm_data()`.
*   The amplitude of the pulse in `mV`, via `get_wfm_data_mV()`.
Both are measures of the total amount of light collected by a PMT.





In [ ]:
# extract datatime from a str. The datetime must follow the fixed format: YYmmddTHHMM
# this is necessary to read the metadata included in the file
def extract_datetime_from_str(s):

    match = re.search('\d{6}T\d{4}', s)
    if match:
        try:
            dt = datetime.strptime(match.group(), '%y%m%dT%H%M')
            return dt
        except ValueError:
            print('Fail finding the datetime string from path: %s' % s)

Let's now define the class that allows to quickly read out a file:

In [ ]:
# this is the class that allows to read a file.
# it stores several parameters related to the detector configuration
# as well as running the function that extracts the data.

# Inputs:
#      raw_data_path (str): path to the raw root file
#      yaml_path (str): path to the configuration yaml file

class WaveformExtractor():
    def __init__(self, raw_data_path, yaml_path='yaml/config.yaml'):

        self.cfg = YamlReader(yaml_path) # load yaml config
        self.if_path = raw_data_path # get the file path

        # load metadata and calibration file
        # --> the are interal class functions (see below)
        self.load_run_info()
        self.load_pmt_info()

        # get daq tree via uproot
        f = uproot.open(raw_data_path)

        # get data from daq tree
        self.n_entries = f['daq'].num_entries

        # set configurations for waveform analysis
        self.wfm = Waveform(self.cfg)
        self.wfm.ch_names=self.ch_names # channel names
        self.wfm.ch_id = self.ch_id # channel ids
        self.wfm.n_boards = self.n_boards # number of boards
        self.wfm.spe_mean = self.spe_mean # single photoelectron response

        self.daq = f['daq'].arrays()
        self.entry_id = 0

    # ---------------
    # this function reads run info metadata
    # finds out datetime and which channels were on.
    def load_run_info(self):

        f = uproot.open(self.if_path)
        dt = extract_datetime_from_str(self.if_path)
        a = f['run_info'].arrays(['n_boards', 'active_ch_id', 'n_event_proc'], library='np')
        self.n_boards = uint16(a['n_boards'][0])
        self.n_event_proc = uint32(a['n_event_proc'][0])
        tmp = a['active_ch_id'][0]
        if isscalar(tmp): # if only 1 active channels, tmp is a scalar and sort will fail
            tmp = [tmp]
        self.ch_id = sorted(uint16(tmp))
        self.ch_names = ["adc_b%d_ch%d" % (i // 100, i % 100) for i in self.ch_id]
        self.ch_name_to_id_dict = dict(zip(self.ch_names, self.ch_id))

    # ---------------
    # the next two functions load up the correct calibration parameters
    # calibration is needed to be able to determine the number of photoelectrons
    # we calibrate the single photoelectron (spe) response of each PMT
    def load_pmt_info(self):

        self.spe_mean = {}
        fpath = CALIB_FILE # read the calibration file
        self._set_spe_result(fpath)

    def _set_spe_result(self, fpath):
        try:
            df = pd.read_csv(fpath)
            df.set_index('ch_name', inplace=True)
            self.spe_fit_results  = df # to be saved in root
            ch_names = df.index
            for ch in ch_names:
                self.spe_mean[ch] = float(df['spe_mean'][ch])
        except:
            sys.exit("your spe_fit_results_file cannot be loaded properly!")

    # ---------------
    # This function actually process one entry (event) of the file, given its index
    # it is called in a loop over the entire entries bt the following function
    def process_one(self, i):

        self.event_id = self.daq[i].event_id
        self.event_ttt = self.daq[i].event_ttt
        self.wfm.reset()
        self.wfm.set_raw_data(self.daq[i])
        self.wfm.find_saturation()
        self.wfm.subtract_flat_baseline()
        self.wfm.do_spe_normalization()
        self.wfm.define_trigger_position()
        self.wfm.correct_daisy_chain_trg_delay()
        self.wfm.sum_channels()
        self.wfm.define_time_axis()
        self.wfm.integrate_waveform()
        self.wfm.calc_roi_info()
        self.wfm.calc_aux_ch_info()

    def process_next(self):

        if self.entry_id<self.n_entries:
            i = self.entry_id
            self.process_one(i)
            self.entry_id+=1
            return True
        else:
            return False

    # ---------------
    # These last few functions are actually extracting data that we will be using
    # for the event display, in particular the total charge of the pulse (in pe)
    # this can be done for a specific input time interval and channel

    # ch_name: channel names identical to how it's defined in DROP.
    #    Options: sum_bot, sum_side, sum_row1,..., sum_row4, ..., sum_col1,..., sum_col4.
    #    Individual channels are accessible too. For example, adc_b1_ch3 is board 1 channel 3.
    def get_wfm_data(self, t_min=370, t_max=470, ch_name='sum_bot'):

        t = self.wfm.time_axis_ns
        mask = (t>=t_min) & (t<t_max)
        x_data=t[mask]
        y_data=self.wfm.amp_pe[ch_name][mask]
        return x_data, y_data

    def get_wfm_data_mv(self, t_min=370, t_max=470, ch_name='sum_bot'):

        t = self.wfm.time_axis_ns
        x_data=t
        y_data=self.wfm.amp_mV[ch_name]

        return x_data, y_data

    def get_aux_ch_mV(self, ch_name='adc_b1_ch0'):
        return self.wfm.aux_ch_area_mV[ch_name]

## Step 3: Preparing the display
Let's the prepare a function that displays data.

First, we define the `(x,y,z)` position of all PMTs in the detector. There are PMTs on the bottom and PMTs on the side (barrell) of the detector.

In [ ]:
# bottom PMTs
locx_bot = [381, 381, 381, 381,
            190.5, 190.5, 190.5, 190.5, 190.5, 190.5, 190.5,
            0, 0, 0, 0, 0, 0, 0, 0,
           -190.5, -190.5, -190.5, -190.5, -190.5, -190.5, -190.5,
           -381, -381, -381, -381]
locy_bot = [-171.45, -57.15, 57.15, 171.45,
           -342.9, -228.6, -114.3, 0, 114.3, 228.6, 342.9,
           -400.05, -285.75, -171.45, -57.15,57.15, 171.45, 285.75, 400.05,
           -342.9, -228.6, -114.3, 0, 114.3, 228.6, 342.9,
           -171.45, -57.15, 57.15, 171.45]

# barrell PMTs
locx_side= [-532.955, -532.955,-532.955,-532.955,
               532.955,532.955,532.955,532.955,
               0,0,0,0,0,0,0,0,]
locy_side= [0,0,0,0,0,0,0,0,
               -532.955, -532.955,-532.955,-532.955,
               532.955,532.955,532.955,532.955]
locz_side= [-495.3, -336.55, 222.25, 393.7,
               -495.3, -336.55, 222.25, 393.7,
               -495.3, -336.55, 222.25, 393.7,
               -495.3, -336.55, 222.25, 393.7]

We can now make a function that displays the PMTs, with a circle that is proportional to the amount of charge they collected.
We can assume to have an array, `chg`, that contains the total charge for each PMT channel id, and use it as input.

Since PMTs are either on the bottom or on the barrell, we can make two 2D plots for each case.

In [ ]:
def display_charge(chg):
    dis_x=[]
    dis_y=[]
    area=[]
    area2=[]

    # setup two plots
    plt.rcParams['figure.figsize'] = [15, 8]
    fig, (ax1, ax2) = plt.subplots(1, 2)

    # for each PMT, make the area proportional to the charge
    # if it's a barrell PMT, use an angle instead of x,y
    for i in range(len(chg)):
        if i < 30:
            area.append(chg[i]*150)
        elif i < 46:
            if i>29 and i<34:
                dis_x.append(0)
            if i>33 and i<38:
                dis_x.append(180)
            if i>37 and i<42:
                dis_x.append(90)
            if i>41 and i<46:
                dis_x.append(270)
            dis_y.append(locz_side[i-30])
            area2.append(chg[i-30]*150)

    # plot the PMTs (just their positions)
    ax1.scatter(locx_bot, locy_bot, c='black', s=50, alpha=0.3)
    ax2.scatter(dis_x, dis_y, c='black', s=50, alpha=0.3)

    # plots the PMTs (heatmaps)
    ax1.scatter(locx_bot, locy_bot, s=area, c=area, alpha=1, label="Bottom plane",cmap='YlOrRd')
    ax2.scatter(dis_x, dis_y, s=area2, c=area2, alpha=1, label="Barrel plane",cmap='YlOrRd')

    # add labels and legends
    ax1.set_xlabel("X (mm)")
    ax1.set_ylabel("Y (mm)")
    ax2.set_xlabel("$\phi$ (deg.) with R = 533 mm")
    ax2.set_ylabel("Z (mm)")
    ax1.legend(loc='upper right',fontsize=12)
    ax2.legend(loc='center right',fontsize=12)
    ax1.grid()
    ax2.grid()
    plt.show()

On top of 2D plots, we can also attemp to get a full 3D event display. Apart from placing the PMTs, we can also try to add a cylinder to represent the detector volume.

In [ ]:
# define a cylinder by its center, radius and height
# this function extracts points on the cylinder surface
def cylinder_along_z(center_x,center_y,radius,height_z):
    z = np.linspace(-height_z/2., height_z/2., 50)
    theta = np.linspace(0, 2*np.pi, 50)
    theta_grid, z_grid=np.meshgrid(theta, z)
    x_grid = radius*np.cos(theta_grid) + center_x
    y_grid = radius*np.sin(theta_grid) + center_y
    return x_grid,y_grid,z_grid

In [ ]:
def display_3d(chg):

    area=[]
    area2=[]

    # setup two plots
    fig = plt.figure(figsize=(15, 8))
    ax = fig.add_subplot(projection='3d')

    # for each PMT, make the area proportional to the charge
    for i in range(len(chg)):
      if i < 30:
        area.append(chg[i]*150)
      elif i < 46:
        area2.append(chg[i-30]*150)

    # plot the PMTs (just their positions)
    ax.scatter(locx_bot, locy_bot, -600., c='black', s=50, alpha=0.2)
    ax.scatter(locx_side, locy_side, locz_side, c='black', s=50, alpha=0.4)

    # plot the PMTs (heatmaps)
    ax.scatter(locx_bot, locy_bot, -600., c=area, s=area,cmap='YlOrRd')
    ax.scatter(locx_side, locy_side, locz_side, c=area2, s=area2, cmap='YlOrRd')

    xc,yc,zc = cylinder_along_z(0.,0.,533.,1100)
    ax.plot_surface(xc, yc, zc, alpha=0.2)

    # add labels and legends
    ax.set_xlabel("X (mm)")
    ax.set_ylabel("Y (mm)")
    ax.set_zlabel("Z (mm)")

    fig.tight_layout()

## Step 4: Processing and plotting
Now all the ingredients are ready and it is possible to process the input file using the class `WaveformExtractor` and feed its output to the plotting function.
We can package this processing into another class (`processor`), which makes it easier to access multiple events or files.

We will need to specify which channels we want to read out, so we begin by creating a string containing *all* channel names.

In [ ]:
# channel names depend on the readout board (b1 to b4)
# each DAQ board hosts 16 channels max
ch_string = ""
for i in range(15):
    ch_string += "adc_b1_ch{} ".format(i+1)
for i in range(15):
    ch_string += "adc_b2_ch{} ".format(i)
for i in range(16):
    ch_string += "adc_b3_ch{} ".format(i)
for i in range(12):
    ch_string += "adc_b4_ch{} ".format(i)

We define a class to process and plot events. Events are sequential entries in the file, so it is possible to process many of them sequentially. The main function `process_data` has three arguments:
* the first entry (event) to be process;
* the number of entries (events) to process;
* an optional flag to plot raw waveforms.

So for example, setting the first entry to `2` and the number of entries to `10` will process event numbers `2`, ..., `11`.

In [ ]:
# defining a class to process events
class processor():
    def __init__(self):

        # use WaveformExtractor to read out the file
        self.wf = WaveformExtractor(FILE_PATH, CONFIG_PATH)
        print("processor ready")

    # this is the main function of the class
    # it takes three inputs as describe before
    def process_data(self, sevt, nevts, show=False):

        tmin = 220
        tmax = 320
        tt = []
        tt_mv1 = []

        # look at entries from sevt to nevts-1
        for ii in range(sevt,sevt+nevts):
          # process the entry
          self.wf.process_one(ii)
          event_id=self.wf.event_id
          event_ttt=self.wf.event_ttt

          evt_chg=[] # prepare to save the charge data
          chn_count = 0

          # get the data between t_min and t_max
          # looping through all channels defined before
          for ich in ch_string.split():
            if chn_count>45:
              break
            if "adc" in ich:
              chn_count += 1
              # use WaveformExtractor function to get charge
              x_data, y_data = self.wf.get_wfm_data(t_min=0, t_max=2000, ch_name='{}'.format(ich))
            else:
              print ("can't do this channel")

            yy_data = y_data.tolist()
            aa = yy_data.index(np.max(yy_data))

            tmin = x_data[aa] - 20
            tmax = x_data[aa] + 40

            if np.max(y_data) > -1e9:
              if "adc" in ich:
                x_data_mv, y_data_mv = self.wf.get_wfm_data(t_min=tmin, t_max=tmax, ch_name='{}'.format(ich))
              else:
                print ("can't do this channel")
              summ_mv = 0
              for idata in range(len(y_data_mv)):
                summ_mv += y_data_mv[idata]*2
              evt_chg.append(summ_mv)
              if show and chn_count<10:
                plt.subplot(3,3,chn_count)
                plt.plot(x_data_mv, y_data_mv, color='k', label='channel %s'%ich)
              if show:
                plt.legend()

          # evalute if it is crossing all the way
          if (self.wf.get_aux_ch_mV(ch_name='adc_b1_ch0') + self.wf.get_aux_ch_mV(ch_name='adc_b2_ch15')>300 ):
            print ("this is a crossing muon.")
          else:
            print ("this is not a crossing muon.")

          display_charge(evt_chg)
          display_3d(evt_chg)

Initialize the processor:

In [ ]:
pr = processor()

You can now plot the events by choosing the entries you want to display:

In [ ]:
start_entry = 0
n_entries = 1
show_wf=False

# arguments: first entry, number of events, show waveform
result = pr.process_data(start_entry, n_entries, show_wf)

Try to go through some of the events you collected. Do you notice anything interesting? Do you see any Cherenkov rings? Do you see a muon crossing the entire detector?